# L1C Formatting Eploration

In [1]:
import pandas as pd
import h5py
from pyhdf.SD import SD, SDC
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files')
# import icare
import pickle
import copy
os.getcwd()

'C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files'

In [ ]:
os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files')
session = icare.ICARESession('GRASP Files/PARASOL')

ICARE Username: sjw5ke@virginia.edu


In [3]:
path='PARASOL/L1_B-HDF.v1.00/2008/2008_06_01'
file_list = list(session.listdir(path))
file_path = path + '/' + file_list[0]
session.get_file(file_path)

'GRASP Files/PARASOL\\PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5'

In [2]:
f = h5py.File("GRASP Files\\PARASOL\\PARASOL\\L1_B-HDF.v1.00\\2008\\2008_06_01\\POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5")

In [10]:
list(f['Data_Directional_Fields']['thetas'].attrs)

['DIMENSION_LABELS',
 'HDF_Calibration_Equation',
 'Num_Fill',
 'Num_Valid',
 'Num_missing_value',
 '_FillValue',
 'actual_range',
 'add_offset',
 'add_offset_err',
 'long_name',
 'missing_value',
 'scale_factor',
 'scale_factor_err',
 'units']

In [36]:
print("Solar Zenith:", f['Data_Directional_Fields']['thetas'].attrs['long_name'])
print("Sensor Zenith:", f['Data_Directional_Fields']['thetav'].attrs['long_name'])
print("Relative Azimuth:", f['Data_Directional_Fields']['phi'].attrs['long_name'])

Solar Zenith: solar zenith angle of the pixel center 
Sensor Zenith: view zenith angle of the pixel center  for filter 670P2
Relative Azimuth: relative azimuth angle of the pixel center  for filter 670P2


if solar zenith is at 90 degrees - horizon

not interested in angles that are > 60 

only want < 60 degrees

Maybe just take a subset of pixels (just the middle 100 longitudes, all the latitudes)

In [35]:
np.unique(np.array(f['Geolocation_Fields']['Latitude'])[1000])

array([-9.99990000e+04,  3.44166667e+01])

In [27]:
np.array(f['Data_Directional_Fields']['phi']).shape

(3240, 6480, 16)

In [11]:
thetas = np.array(f['Data_Directional_Fields']['thetas'])
print(thetas.shape)

(3240, 6480, 16)


In [12]:
thetas[0][0]

array([65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
       65535, 65535, 65535, 65535, 65535, 65535, 65535], dtype=uint16)

In [13]:
thetav = np.array(f['Data_Directional_Fields']['thetav'])
print(thetav.shape)

(3240, 6480, 16)


In [14]:
thetav[0][0]

array([65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535, 65535,
       65535, 65535, 65535, 65535, 65535, 65535, 65535], dtype=uint16)

# Create example file from h5

In [5]:
measurement_dict = {}
for cat in ['I_np','I_p','Q','U']:
    print(cat)
    
    if cat == 'I_np':
        tag = cat.replace('_np','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field)])
    else:
        tag = cat.replace('_p','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field) and ('NP' not in field)])

    print("tag:", tag)
    fields.sort()
    print(fields)
    arrays = []
    scales = []
    long_names = []
    fills = []
    units = []
    for field in fields:
            
        print(field, end=", ")
        
        scales.append(f['Data_Directional_Fields'][field].attrs['scale_factor'])
        long_names.append(f['Data_Directional_Fields'][field].attrs['long_name'])
        fills.append(f['Data_Directional_Fields'][field].attrs['_FillValue'])
        units.append(f['Data_Directional_Fields'][field].attrs['units'])

        arrays.append(np.array(f['Data_Directional_Fields'][field]))
        
    if len(np.unique(scales)) == 1:
        scales = scales[0]
    if len(np.unique(fills)) == 1:
        fills = fills[0]
    if len(np.unique(units)) == 1:
        units = units[0]
    
    measurement_dict[cat] = {}
    measurement_dict[cat]['fields'] = fields
    measurement_dict[cat]['scale'] = scales
    measurement_dict[cat]['long_name'] = long_names
    measurement_dict[cat]['fill'] = fills
    measurement_dict[cat]['units'] = units
    measurement_dict[cat]['data'] = np.stack(arrays,axis=3)

I_np
tag: I
['I1020NP', 'I443NP', 'I490P', 'I565NP', 'I670P', 'I763NP', 'I765NP', 'I865P', 'I910NP']
I1020NP, I443NP, I490P, I565NP, I670P, I763NP, I765NP, I865P, I910NP, I_p
tag: I
['I490P', 'I670P', 'I865P']
I490P, I670P, I865P, Q
tag: Q
['Q490P', 'Q670P', 'Q865P']
Q490P, Q670P, Q865P, U
tag: U
['U490P', 'U670P', 'U865P']
U490P, U670P, U865P, 

# Create Final Dictionary and Populate

In [6]:
final_dict = {}
final_dict['observation_data'] = {}
final_dict['observation_data']['I_PARASOL'] = copy.deepcopy(measurement_dict['I_np'])

## Wavelengths

In [7]:
def add_wavelengths(polarized=True):
    if polarized:
        print("polarized")
        tag = 'I_p'
        field_name = 'polarization_wavelengths'
        lambdas = [int(field.replace('I','').replace('P','')) for field in measurement_dict[tag]['fields']]

    else:
        print("non-polarized")
        tag = 'I_np'
        field_name = 'intensity_wavelengths'
        lambdas = [int(field.replace('I','').replace('NP','').replace('P','')) for field in measurement_dict[tag]['fields']]

    shape = measurement_dict[tag]['data'].shape
    new_shape = []

    lambda_arrs = []

    for lam in lambdas:    
        new_shape = []
        for dim in shape[:-1]:
            new_shape.append(dim)
        # new_shape.append(1)

        lambda_arrs.append(np.full(new_shape, fill_value = np.full(16,lam)))
        # break

    full_lambdas_arr = np.stack(lambda_arrs,axis=3)
    print(full_lambdas_arr.shape)
    
    final_dict['sensor_views_bands'] = {}
    final_dict['sensor_views_bands'][field_name] = {}
    final_dict['sensor_views_bands'][field_name]['scale'] = 1
    final_dict['sensor_views_bands'][field_name]['long_name'] = 'field_name'
    final_dict['sensor_views_bands'][field_name]['fill'] = 32767
    final_dict['sensor_views_bands'][field_name]['units'] = 'tbd'
    final_dict['sensor_views_bands'][field_name]['data'] = full_lambdas_arr
    
for i in [True,False]:
    add_wavelengths(i)

polarized
(3240, 6480, 16, 3)
non-polarized
(3240, 6480, 16, 9)


In [8]:
measurement_dict.keys()

dict_keys(['I_np', 'I_p', 'Q', 'U'])

## DOLP
* Calculate DOLP matrix
    * DOLP = sqrt(Q^2 + U^2)/I

In [9]:
scale = measurement_dict['Q']['scale']
scale
fill = measurement_dict['Q']['fill']
fill

32767

In [10]:
# scale, abs
for key in measurement_dict.keys():
    if key == 'I_p':
        I_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        I_arr[np.abs(I_arr) == fill] = 1
    
    elif key == 'Q':
        Q_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        Q_arr[np.abs(Q_arr) == fill] = 1
        
    elif key == 'U':
        U_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        U_arr[np.abs(U_arr) == fill] = 1
        
    else:
        continue

MemoryError: Unable to allocate 7.51 GiB for an array with shape (3240, 6480, 16, 3) and data type float64

In [ ]:
DOLP_arr_unfltrd = np.divide(np.sqrt(np.add(np.square(Q_arr), np.square(U_arr))), I_arr)

In [ ]:
# rename - DOLP
DOLP_arr_unfltrd[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

In [ ]:
indx = np.where((np.abs(I_arr) != (fill * scale)) & (np.abs(Q_arr) != (fill * scale)) & (np.abs(U_arr) != (fill * scale)))

In [ ]:
# add to dictionary

final_dict['observation_data']['DOLP_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['DOLP_PARASOL']['scale'] = 1
final_dict['observation_data']['DOLP_PARASOL']['long_name'] = 'INSERT LONG NAME'
final_dict['observation_data']['DOLP_PARASOL']['fill'] = fill
final_dict['observation_data']['DOLP_PARASOL']['units'] = 'INSERT UNITS'
final_dict['observation_data']['DOLP_PARASOL']['data'] = DOLP_arr_unfltrd

## Q over I, U over I

In [ ]:
Q_over_I = np.divide(Q_arr, I_arr)
U_over_I = np.divide(U_arr, I_arr)

Q_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

U_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

final_dict['observation_data']['Q_over_I_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['Q_over_I_PARASOL']['scale'] = 1
final_dict['observation_data']['Q_over_I_PARASOL']['long_name'] = 'INSERT LONG NAME'
final_dict['observation_data']['Q_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['Q_over_I_PARASOL']['units'] = 'INSERT UNITS'
final_dict['observation_data']['Q_over_I_PARASOL']['data'] = Q_over_I

final_dict['observation_data']['U_over_I_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['U_over_I_PARASOL']['scale'] = 1
final_dict['observation_data']['U_over_I_PARASOL']['long_name'] = 'INSERT LONG NAME'
final_dict['observation_data']['U_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['U_over_I_PARASOL']['units'] = 'INSERT UNITS'
final_dict['observation_data']['U_over_I_PARASOL']['data'] = U_over_I

# Angles, Altitude, & Location

In [42]:
f['Geolocation_Fields'].keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'column_number', 'land_sea_flag', 'row_number', 'surface_altitude']>

In [ ]:
for field in ['thetas','thetav','phi']:
    if field == 'thetas':
        tag = 'solar_zenith'
    elif field == 'thetav':
        tag = 'sensor_zenith'
    elif field == 'phi':
        tag = 'relative_azimuth'
        
    final_dict['geolocation_data'][tag] = {}
    final_dict['geolocation_data'][tag]['scale'] = f['Data_Directional_Fields'][field].attrs['scale_factor']
    final_dict['geolocation_data'][tag]['long_name'] = f['Data_Directional_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['fill'] = f['Data_Directional_Fields'][field].attrs['_FillValue']
    final_dict['geolocation_data'][tag]['units'] = f['Data_Directional_Fields'][field].attrs['units']
    final_dict['geolocation_data'][tag]['data'] = np.array(f['Data_Directional_Fields'][field])

In [ ]:
for field in ['Latitude','Longitude','surface_altitude']:
    if field == 'surface_altitude':
        tag = 'altitude'
    else:
        tag = field
        
    final_dict['geolocation_data'][tag] = {}
    final_dict['geolocation_data'][tag]['scale'] = f['Geolocation_Fields'][field].attrs['scale_factor']
    final_dict['geolocation_data'][tag]['long_name'] = f['Geolocation_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['fill'] = f['Geolocation_Fields'][field].attrs['_FillValue']
    final_dict['geolocation_data'][tag]['units'] = f['Geolocation_Fields'][field].attrs['units']
    final_dict['geolocation_data'][tag]['data'] = np.array(f['Geolocation_Fields'][field])

In [ ]:
with open('final_dict.pickle', 'w') as f:
    pickle.dump(final_dict, f)